In [ ]:
import bw2data as bd
import bw2calc as bc
import multifunctional as mf

# Create a basic database

Note the node type `multifunctional` and the edge attribute `functional`.

In [2]:
if "multifunctional-example" in bd.projects:
    bd.projects.delete_project("multifunctional-example", True)

In [3]:
bd.projects.set_current("multifunctional-example")

In [4]:
BASIC_DATA = {
    ("basic", "a"): {
        "name": "flow - a",
        "code": "a",
        "unit": "kg",
        "type": "emission",
        "categories": ("air",),
    },
    ("basic", "1"): {
        "name": "process - 1",
        "code": "1",
        "location": "first",
        "type": "multifunctional",
        "exchanges": [
            {
                "functional": True,
                "type": "production",
                "name": "first product - 1",
                "unit": "kg",
                "amount": 4,
                "properties": {
                    "price": 7,
                    "mass": 6,
                },
            },
            {
                "functional": True,
                "type": "production",
                "name": "second product - 1",
                "unit": "megajoule",
                "amount": 6,
                "properties": {
                    "price": 12,
                    "mass": 4,
                },
            },
            {
                "type": "biosphere",
                "name": "flow - a",
                "amount": 10,
                "input": ("basic", "a"),
            },
        ],
    },
}

db = mf.MultifunctionalDatabase("basic")
db.register(default_allocation="price")
db.write(BASIC_DATA)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 5698.78it/s]
2024-07-07 08:22:05.141 | DEBUG    | multifunctional.node_classes:allocate:68 - Allocating Multifunctional: 'process - 1' (None, first, None) (id: 2) with strategy price
2024-07-07 08:22:05.142 | DEBUG    | multifunctional.allocation:generic_allocation:56 - Using allocation factor 0.28 for functional edge {'functional': True, 'type': 'production', 'name': 'first product - 1', 'unit': 'kg', 'amount': 4, 'properties': {'price': 7, 'mass': 6}, 'input': ('basic', '1'), 'mf_artificial_code': True, 'output': ('basic', '1')} on activity {'name': 'process - 1', 'code': '1', 'location': 'first', 'type': 'multifunctional', 'database': 'basic', 'id': 2, 'exchanges': [{'functional': True, 'type': 'production', 'name': 'first product - 1', 'unit': 'kg', 'amount': 4, 'properties': {'price':

Vacuuming database 


Writing the database also allocates the multifunctional process:

In [5]:
for node in bd.Database("basic"):
    print(node, node['type'])

Read-only allocated process: 'process - 1' (kg, first, None) readonly_process
Multifunctional: 'process - 1' (None, first, None) multifunctional
Read-only allocated process: 'process - 1' (megajoule, first, None) readonly_process
'flow - a' (kg, None, ('air',)) emission


Allocated processes are read-only

In [6]:
attributes = {"reference product": "first product - 1", "database": "basic"}
node = bd.get_node(**attributes)
node

Read-only allocated process: 'process - 1' (kg, first, None)

In [7]:
node['foo'] = 'bar'

NotImplementedError: This node is read only. Update the corresponding multifunctional process.

But they can be used in LCIA:

In [8]:
m = bd.Method(("foo",))
m.register()
m.write([(bd.get_node(code="a").id, 5)])

In [9]:
fu, objs, _ = bd.prepare_lca_inputs(
    demand={node: 1}, method=("foo",)
)
lca = bc.LCA(fu, data_objs=objs)
lca.lci()
lca.lcia()
lca.score

3.4999999403953552